In [16]:
from google.colab import drive
drive.mount("/gdrive")
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [17]:
cd /gdrive/My Drive/renderresults

/gdrive/My Drive/renderresults


In [18]:
!ls

blender2.83  blender2.91.tar.xz     output000-250.png.mp4  setgpu.py
blender2.91  output000-002.png.mp4  output003.png


In [19]:
#@title Setup
#@markdown Please configure your setup

blender_version = 'blender2.91' #@param ["blender2.91", "blender2.81", "blender2.80"] {allow-input: false}
path_to_blend = 'fire.blend'  #@param {type: "string"}
output_path = 'renderresults/output###.png'  #@param {type: "string"}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}
#@markdown ---

In [20]:
if blender_version == "blender2.91":
    download_path="https://download.blender.org/release/Blender2.91/blender-2.91.0-linux64.tar.xz"
elif blender_version == "blender2.81":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.81/blender-2.81-linux-glibc217-x86_64.tar.bz2"
elif blender_version == "blender2.82":
    download_path="https://ftp.halifax.rwth-aachen.de/blender/release/Blender2.82/blender-2.82-linux64.tar.xz"

In [21]:
!mkdir $blender_version

if blender_version == "blender2.91":
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1
else:
    !wget -O '{blender_version}.tar.bz2' -nc $download_path
    !tar -xf '{blender_version}.tar.bz2' -C ./$blender_version --strip-components=1

mkdir: cannot create directory ‘blender2.91’: File exists
File ‘blender2.91.tar.xz’ already there; not retrieving.
^C


In [22]:
import os

os.environ["LD_PRELOAD"] = ""

!apt update
!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4
os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

!echo $LD_PRELOAD

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 http://archive.ubuntu.co

In [23]:

!apt install libboost-all-dev
!apt install libgl1-mesa-dev
!apt install libglu1-mesa libsm-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libglu1-mesa is already the newest version (9.0.0-2.1build1).
libsm-dev is already the newest version (2:1.2.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [24]:
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "# Attempt to set GPU device types if available\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "#for scene in bpy.data.scenes:\n"+\
    "#    scene.render.tile_x = 64\n"+\
    "#    scene.render.tile_y = 64\n"+\
    "# Enable all CPU and GPU devices\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

In [ ]:
!sudo ./$blender_version/blender -P './setgpu.py'  -b '/gdrive/My Drive/{path_to_blend}' -E CYCLES -o '/gdrive/My Drive/{output_path}' -s 1 -e 250 -a

Streaming output truncated to the last 5000 lines.
Fra:36 Mem:398.72M (Peak 501.53M) | Time:00:02.89 | Remaining:00:12.16 | Mem:79.52M, Peak:81.58M | Scene, View Layer | Rendered 362/2040 Tiles, Sample 32/32, Denoised 300 tiles
Fra:36 Mem:398.72M (Peak 501.53M) | Time:00:02.90 | Remaining:00:12.19 | Mem:79.52M, Peak:81.58M | Scene, View Layer | Rendered 362/2040 Tiles, Sample 32/32, Denoised 301 tiles
Fra:36 Mem:399.22M (Peak 501.53M) | Time:00:02.90 | Remaining:00:12.16 | Mem:80.02M, Peak:81.58M | Scene, View Layer | Rendered 363/2040 Tiles, Sample 32/32, Denoised 301 tiles
Fra:36 Mem:398.72M (Peak 501.53M) | Time:00:02.91 | Remaining:00:12.19 | Mem:80.94M, Peak:81.58M | Scene, View Layer | Rendered 363/2040 Tiles, Sample 32/32, Denoised 302 tiles
Fra:36 Mem:399.22M (Peak 501.53M) | Time:00:02.91 | Remaining:00:12.15 | Mem:81.44M, Peak:81.58M | Scene, View Layer | Rendered 364/2040 Tiles, Sample 32/32, Denoised 302 tiles
Fra:36 Mem:398.72M (Peak 501.53M) | Time:00:02.91 | Remaining:00